In [1]:
import tensorflow as tf

In [2]:
import numpy as np

## GRAPH

In [59]:
tf.reset_default_graph()

### Model

If we want to be able to import the graph tensors and restore the trained variables in another session, we should name every single piece of the graph.

I would love that naming the inputs `inputs` and the logits `logits` was a convention, so all the models were just black boxes.

In [60]:
x = tf.placeholder(tf.float32, [None, 2], name='inputs')

In [61]:
y = tf.placeholder(tf.float32, [None, 1], name='targets')

In [62]:
with tf.name_scope('first_layer'):
    W = tf.Variable(tf.truncated_normal([2, 1], stddev=0.05), name='W')
    b = tf.Variable(tf.random_normal([1]), name='b')

    output = tf.add(tf.matmul(x, W), b, name='output')

In [63]:
logits = tf.identity(output, name='logits')

### Training

In [64]:
learning_rate = 0.00001 # Terribly important hyperparameter. It can make your net go totally crazy.

In [65]:
with tf.name_scope('training'):
    cost = tf.reduce_sum(tf.square(logits - y))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

### Accuracy

In [66]:
with tf.name_scope('accuracy'):
    accuracy = tf.reduce_mean(y - tf.abs(logits - y)) / tf.reduce_mean(y)

## DATA

In [67]:
from helper import get_data

inputs, targets = get_data(max_int=10, size=10000)

In [68]:
from helper import split_data

train_inputs, test_inputs, train_targets, test_targets = split_data(inputs, targets)

## SESSION

In [69]:
epochs = 2

In [70]:
from helper import get_batches

batch_size = 1000

In [71]:
# Create a saver object so we can save the graph and variable values.
saver = tf.train.Saver()

In [72]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # TRAINING
    for epoch in range(epochs):

        batch_x, batch_y = get_batches(train_inputs, train_targets, batch_size)

        for batch in range(train_inputs.shape[0]//batch_size):

            sess.run(optimizer, feed_dict={
                x: batch_x[batch],
                y: batch_y[batch]
                })

            train_loss = sess.run(cost, feed_dict={
                x: batch_x[batch],
                y: batch_y[batch]
                })

            print('Epoch {:>2}, Batch {:3} - '
                'Training Loss: {:>10.4f}'.format(
                    epoch + 1, 
                    batch + 1,
                    train_loss))
    

    # TESTING
    test_accuracy = sess.run(accuracy, feed_dict={
        x: test_inputs,
        y: test_targets
        })
    print('\nTesting Accuracy: {}'.format(test_accuracy))

    final_test = sess.run(logits, feed_dict={
        x: np.array([[5, 7]])
        })
    print("\nThe sum of 5 plus 7 is {}".format(final_test[0][0])) # The result will be near 12.

    print("\nThe weights are: {}".format(sess.run(W)))
    print("and the bias is: {}".format(sess.run(b)))
    
    # SAVING
    # Save the graph and variables to disk.
    # At first it will be saved in a temporal directory. If we are satisfied with the results, 
    # we should save all three checkpoint files in a safer directory.
    save_path = saver.save(sess, "/tmp/regression_model.ckpt")
    print("\nModel saved in file: %s" % save_path)

Epoch  1, Batch   1 - Training Loss:  5221.4307
Epoch  1, Batch   2 - Training Loss:   334.3157
Epoch  1, Batch   3 - Training Loss:    65.3385
Epoch  1, Batch   4 - Training Loss:    40.1266
Epoch  1, Batch   5 - Training Loss:    36.2257
Epoch  1, Batch   6 - Training Loss:    35.3869
Epoch  1, Batch   7 - Training Loss:    34.5284
Epoch  2, Batch   1 - Training Loss:    32.7147
Epoch  2, Batch   2 - Training Loss:    30.2309
Epoch  2, Batch   3 - Training Loss:    32.1030
Epoch  2, Batch   4 - Training Loss:    30.6360
Epoch  2, Batch   5 - Training Loss:    30.1663
Epoch  2, Batch   6 - Training Loss:    29.5621
Epoch  2, Batch   7 - Training Loss:    29.6601

Testing Accuracy: 0.985505223274231

The sum of 5 plus 7 is 12.000414848327637

The weights are: [[ 0.96216393]
 [ 0.96657526]]
and the bias is: [ 0.42356801]
Model saved in file: /tmp/regression_model.ckpt
